# Assignment 2 - Task 1: Data Importing & Transformation

**Course:** CS424 - Visualization & Visual Analytics (Fall 2025)  
**Dataset:** San Francisco Assessor Historical Secured Property Tax Rolls  
**Source:** https://data.sfgov.org/Housing-and-Buildings/Assessor-Historical-Secured-Property-Tax-Rolls/wv5m-vpq2

---

## Overview

This notebook implements Task 1 of Assignment 2, focusing on:
1. Data importing and loading **from GeoJSON format**
2. Data profiling and statistics computation
3. Data transformation and cleaning
4. Feature engineering and derived columns

The dataset contains property tax assessment records for San Francisco from 2007-2023, with information about property characteristics, values, locations, and ownership.

### Required Libraries

Before running this notebook, ensure you have the following libraries installed:

```bash
pip install pandas numpy matplotlib seaborn
pip install geopandas  # Required for GeoJSON loading
pip install sodapy     # Optional, for SODA API access
pip install pyarrow    # Required for Parquet export
```

## 1. Environment Setup

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import geopandas for GeoJSON loading
try:
    import geopandas as gpd
    print("✓ GeoPandas available")
except ImportError:
    print("⚠ GeoPandas not installed. Install with: pip install geopandas")
    print("  GeoJSON loading will not work without geopandas.")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("\nLibraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Data Importing

### 2.1 Load the Dataset

The dataset is available from San Francisco's Open Data Portal in multiple formats. 
We'll use **GeoJSON** format which includes geographic coordinates for spatial analysis.

In [ ]:
local_file = r"C:\Users\anand\Desktop\SEM 3\CS 424\Project\Assessor_Historical_Secured_Property_Tax_Rolls_20251012.geojson"

gdf = gpd.read_file(local_file)
df = pd.DataFrame(gdf.drop(columns='geometry'))
df['latitude'] = gdf.geometry.y
df['longitude'] = gdf.geometry.x

print(f"✓ Loaded {len(df):,} records")

### 2.2 Initial Data Inspection

In [ ]:
# Display basic information about the dataset
print("="*80)
print("DATA INSPECTION")
print("="*80)
print(f"\nDataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Display first few rows
print("\nFirst 5 rows:")
display(df.head())

# Display column information
print("\nColumn Information:")
display(df.info())

In [ ]:
# Check for missing values
print("Missing Values Analysis:")

missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values(
    'Missing_Percentage', ascending=False
)

if len(missing_summary) > 0:
    display(missing_summary)
else:
    print("No missing values found!")

## 3. Data Profiling

### 3.1 Temporal Analysis

In [ ]:
# Analyze temporal distribution
print("TEMPORAL ANALYSIS")

# Convert year to integer if needed
df['year'] = pd.to_numeric(df['closed_roll_year'], errors='coerce').astype('Int64')

# Filter to relevant years (2015-2023)
df_filtered = df[(df['year'] >= 2015) & (df['year'] <= 2023)].copy()

print(f"\nOriginal dataset: {len(df):,} records")
print(f"Filtered dataset (2015-2023): {len(df_filtered):,} records")

# Records per year
year_counts = df_filtered['year'].value_counts().sort_index()
print("\nRecords per year:")
for year, count in year_counts.items():
    print(f"  {year}: {count:,} properties")

### 3.2 Geographic Analysis

In [ ]:
# Analyze geographic distribution
print("GEOGRAPHIC ANALYSIS")

# Count unique neighborhoods
n_neighborhoods = df_filtered['assessor_neighborhood'].nunique()
print(f"\nTotal neighborhoods: {n_neighborhoods}")

# Top neighborhoods by property count
top_neighborhoods = df_filtered['assessor_neighborhood'].value_counts().head(15)
print("\nTop 15 neighborhoods by property count:")
for idx, (neighborhood, count) in enumerate(top_neighborhoods.items(), 1):
    print(f"  {idx:2d}. {neighborhood}: {count:,}")

### 3.3 Property Characteristics Analysis

In [ ]:
# Analyze property characteristics
print("PROPERTY CHARACTERISTICS")

# Convert numeric columns
numeric_cols = ['number_of_bedrooms', 'number_of_bathrooms', 'number_of_rooms', 
                'number_of_stories', 'property_area', 'year_property_built']

for col in numeric_cols:
    if col in df_filtered.columns:
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

# Bedrooms analysis
print("\nBedrooms:")
print(f"  Mean: {df_filtered['number_of_bedrooms'].mean():.1f}")
print(f"  Median: {df_filtered['number_of_bedrooms'].median():.0f}")
print(f"  Range: {df_filtered['number_of_bedrooms'].min():.0f} - {df_filtered['number_of_bedrooms'].max():.0f}")

bedroom_dist = df_filtered['number_of_bedrooms'].value_counts().sort_index().head(10)
print("\n  Distribution (top 10):")
for bedrooms, count in bedroom_dist.items():
    print(f"    {bedrooms:.0f} bedrooms: {count:,} properties")

# Building age analysis
print("\nBuilding Age:")
print(f"  Oldest: {df_filtered['year_property_built'].min():.0f}")
print(f"  Newest: {df_filtered['year_property_built'].max():.0f}")
print(f"  Median: {df_filtered['year_property_built'].median():.0f}")
print(f"  Mean: {df_filtered['year_property_built'].mean():.0f}")

### 3.4 Financial Analysis

In [ ]:
# Analyze property values
print("FINANCIAL ANALYSIS")
print("="*80)

# Convert financial columns to numeric
value_cols = ['assessed_land_value', 'assessed_improvement_value']
for col in value_cols:
    if col in df_filtered.columns:
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

# Create total assessed value
df_filtered['total_assessed_value'] = (
    df_filtered['assessed_land_value'] + df_filtered['assessed_improvement_value']
)

print("\nTotal Assessed Value Statistics:")
print(f"  Count: {df_filtered['total_assessed_value'].count():,}")
print(f"  Mean: ${df_filtered['total_assessed_value'].mean():,.0f}")
print(f"  Median: ${df_filtered['total_assessed_value'].median():,.0f}")
print(f"  Std Dev: ${df_filtered['total_assessed_value'].std():,.0f}")
print(f"  Min: ${df_filtered['total_assessed_value'].min():,.0f}")
print(f"  Max: ${df_filtered['total_assessed_value'].max():,.0f}")

# Percentiles
percentiles = [10, 25, 50, 75, 90, 95, 99]
print("\nPercentiles:")
for p in percentiles:
    value = df_filtered['total_assessed_value'].quantile(p/100)
    print(f"  {p}th: ${value:,.0f}")

# Land vs Improvement value comparison
print("\nLand vs Improvement Value:")
avg_land = df_filtered['assessed_land_value'].mean()
avg_improvement = df_filtered['assessed_improvement_value'].mean()
print(f"  Average Land Value: ${avg_land:,.0f}")
print(f"  Average Improvement Value: ${avg_improvement:,.0f}")
print(f"  Land % of Total: {avg_land/(avg_land+avg_improvement)*100:.1f}%")

### 3.5 Property Type Analysis

In [ ]:
# Analyze property types
print("PROPERTY TYPE ANALYSIS")

# Property class distribution
property_types = df_filtered['property_class_code_definition'].value_counts().head(10)
print("\nTop 10 Property Classes:")
for idx, (ptype, count) in enumerate(property_types.items(), 1):
    print(f"  {idx:2d}. {ptype}: {count:,}")

# Use code distribution
use_types = df_filtered['use_definition'].value_counts().head(10)
print("\nTop 10 Use Definitions:")
for idx, (use, count) in enumerate(use_types.items(), 1):
    print(f"  {idx:2d}. {use}: {count:,}")

## 4. Data Transformation

### 4.1 Data Cleaning

In [ ]:
# Remove invalid or outlier values
print("DATA CLEANING")

# Store original size
original_size = len(df_filtered)

# Remove properties with zero or negative values
df_clean = df_filtered[
    (df_filtered['total_assessed_value'] > 0) &
    (df_filtered['assessed_land_value'] > 0) &
    (df_filtered['assessed_improvement_value'] >= 0)
].copy()

print(f"\nRecords removed: {original_size - len(df_clean):,}")
print(f"Cleaned dataset size: {len(df_clean):,}")

# Handle outliers (optional - using 99th percentile cap)
value_99th = df_clean['total_assessed_value'].quantile(0.99)
print(f"\n99th percentile value: ${value_99th:,.0f}")
print(f"Properties above 99th percentile: {(df_clean['total_assessed_value'] > value_99th).sum():,}")

### 4.2 Feature Engineering

In [ ]:
# Create derived columns
print("FEATURE ENGINEERING")

# 1. Land value percentage
df_clean['land_value_pct'] = (
    df_clean['assessed_land_value'] / df_clean['total_assessed_value'] * 100
)
print("\n✓ Created: land_value_pct")

# 2. Building age
current_year = 2023
df_clean['building_age'] = current_year - df_clean['year_property_built']
print("✓ Created: building_age")

# 3. Property density (value per sqft)
df_clean['value_per_sqft'] = (
    df_clean['total_assessed_value'] / df_clean['property_area']
).replace([np.inf, -np.inf], np.nan)
print("✓ Created: value_per_sqft")

# 4. Neighborhood label (simplified)
df_clean['neighborhood'] = df_clean['assessor_neighborhood'].str.strip()
print("✓ Created: neighborhood")

# 5. Property category (residential vs commercial)
residential_keywords = ['Dwelling', 'Condominium', 'Apartment', 'Residential', 'Timeshare']
df_clean['is_residential'] = df_clean['property_class_code_definition'].apply(
    lambda x: any(kw in str(x) for kw in residential_keywords) if pd.notna(x) else False
)
print("✓ Created: is_residential")

# 6. Value tier
def assign_value_tier(value):
    if pd.isna(value):
        return 'Unknown'
    elif value < 300000:
        return 'Budget'
    elif value < 800000:
        return 'Mid-range'
    elif value < 2000000:
        return 'High-end'
    else:
        return 'Luxury'

df_clean['value_tier'] = df_clean['total_assessed_value'].apply(assign_value_tier)
print("✓ Created: value_tier")

print(f"\nTotal new features created: 6")
print(f"Final dataset shape: {df_clean.shape}")

### 4.3 Data Type Optimization

In [ ]:
# Optimize data types to reduce memory usage
print("DATA TYPE OPTIMIZATION")

memory_before = df_clean.memory_usage(deep=True).sum() / 1024**2
print(f"\nMemory before optimization: {memory_before:.1f} MB")

# Convert to appropriate types
df_clean['year'] = df_clean['year'].astype('int16')
df_clean['number_of_bedrooms'] = df_clean['number_of_bedrooms'].astype('int8')
df_clean['number_of_bathrooms'] = df_clean['number_of_bathrooms'].astype('float16')
df_clean['is_residential'] = df_clean['is_residential'].astype('bool')

# Convert categorical columns
categorical_cols = ['neighborhood', 'property_class_code_definition', 
                   'use_definition', 'value_tier']
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('category')

memory_after = df_clean.memory_usage(deep=True).sum() / 1024**2
memory_reduction = (memory_before - memory_after) / memory_before * 100

print(f"Memory after optimization: {memory_after:.1f} MB")
print(f"Memory reduction: {memory_reduction:.1f}%")

## 5. Visual Data Summaries with Vega-Lite

Create interactive visualizations using Vega-Lite (via Altair) to explore data distributions and relationships.

### 5.1 Setup Altair (Python API for Vega-Lite)

In [ ]:
import altair as alt

# Configure Altair
alt.data_transformers.enable('default', max_rows=None)  # Handle large datasets
alt.renderers.enable('default')  # Use default renderer

print(f"✓ Altair version: {alt.__version__}")
print(f"✓ Vega-Lite visualizations ready")

# For large datasets, we'll sample for interactive visualizations
# Vega-Lite works best with <5000 rows for interactivity
sample_size = 5000
df_sample = df_clean.sample(n=min(sample_size, len(df_clean)), random_state=42)
print(f"\nCreated sample of {len(df_sample):,} records for interactive visualizations")

### 5.2 Univariate Histograms with Binning

Explore distributions of key numerical variables using interactive histograms.

In [ ]:
# Histogram 1: Total Assessed Value Distribution
# Cap at 99th percentile for better visualization
value_99th = df_clean['total_assessed_value'].quantile(0.99)
df_viz = df_sample[df_sample['total_assessed_value'] <= value_99th].copy()

chart_value_hist = alt.Chart(df_viz).mark_bar(
    color='steelblue',
    opacity=0.7
).encode(
    x=alt.X('total_assessed_value:Q',
            bin=alt.Bin(maxbins=50),
            title='Total Assessed Value ($)',
            axis=alt.Axis(format='$,.0f')),
    y=alt.Y('count()',
            title='Number of Properties'),
    tooltip=[
        alt.Tooltip('total_assessed_value:Q', bin=alt.Bin(maxbins=50), title='Value Range', format='$,.0f'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=600,
    height=300,
    title='Distribution of Total Assessed Values (up to 99th percentile)'
).interactive()

chart_value_hist

In [ ]:
# Histogram 2: Building Age Distribution
chart_age_hist = alt.Chart(df_sample[df_sample['building_age'].notna()]).mark_bar(
    color='purple',
    opacity=0.7
).encode(
    x=alt.X('building_age:Q',
            bin=alt.Bin(maxbins=40),
            title='Building Age (years)',
            scale=alt.Scale(domain=[0, 150])),
    y=alt.Y('count()',
            title='Number of Properties'),
    tooltip=[
        alt.Tooltip('building_age:Q', bin=alt.Bin(maxbins=40), title='Age Range'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=600,
    height=300,
    title='Distribution of Building Ages'
).interactive()

chart_age_hist

In [ ]:
# Histogram 3: Land Value Percentage Distribution
chart_land_pct_hist = alt.Chart(df_sample[df_sample['land_value_pct'].notna()]).mark_bar(
    color='darkred',
    opacity=0.7
).encode(
    x=alt.X('land_value_pct:Q',
            bin=alt.Bin(maxbins=50),
            title='Land Value as % of Total',
            scale=alt.Scale(domain=[0, 100])),
    y=alt.Y('count()',
            title='Number of Properties'),
    tooltip=[
        alt.Tooltip('land_value_pct:Q', bin=alt.Bin(maxbins=50), title='Land % Range', format='.1f'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=600,
    height=300,
    title='Distribution of Land Value Percentage'
).interactive()

chart_land_pct_hist

### 5.3 Boxplots for Comparing Distributions

Use boxplots to compare distributions across categories.

In [ ]:
# Boxplot 1: Property Values by Value Tier
chart_value_boxplot = alt.Chart(df_sample).mark_boxplot(
    size=40
).encode(
    x=alt.X('value_tier:N',
            title='Value Tier',
            sort=['Budget', 'Mid-range', 'High-end', 'Luxury']),
    y=alt.Y('total_assessed_value:Q',
            title='Total Assessed Value ($)',
            scale=alt.Scale(type='log'),
            axis=alt.Axis(format='$,.0f')),
    color=alt.Color('value_tier:N',
                    legend=None,
                    scale=alt.Scale(scheme='category10')),
    tooltip=[
        alt.Tooltip('value_tier:N', title='Tier'),
        alt.Tooltip('total_assessed_value:Q', aggregate='median', title='Median Value', format='$,.0f'),
        alt.Tooltip('total_assessed_value:Q', aggregate='q1', title='Q1', format='$,.0f'),
        alt.Tooltip('total_assessed_value:Q', aggregate='q3', title='Q3', format='$,.0f')
    ]
).properties(
    width=500,
    height=350,
    title='Property Value Distribution by Value Tier (log scale)'
).interactive()

chart_value_boxplot

In [ ]:
# Boxplot 2: Building Age by Property Class (Top 5 types)
top_property_types = df_clean['property_class_code_definition'].value_counts().head(5).index.tolist()
df_top_types = df_sample[df_sample['property_class_code_definition'].isin(top_property_types)]

chart_age_by_type = alt.Chart(df_top_types).mark_boxplot(
    size=30
).encode(
    x=alt.X('property_class_code_definition:N',
            title='Property Type',
            axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('building_age:Q',
            title='Building Age (years)',
            scale=alt.Scale(domain=[0, 150])),
    color=alt.Color('property_class_code_definition:N',
                    legend=None),
    tooltip=[
        alt.Tooltip('property_class_code_definition:N', title='Property Type'),
        alt.Tooltip('building_age:Q', aggregate='median', title='Median Age'),
        alt.Tooltip('count()', title='Sample Count')
    ]
).properties(
    width=600,
    height=350,
    title='Building Age Distribution by Property Type (Top 5)'
).interactive()

chart_age_by_type

In [ ]:
# Boxplot 3: Land Value Percentage by Residential vs Non-Residential
chart_land_by_residential = alt.Chart(df_sample).mark_boxplot(
    size=60
).encode(
    x=alt.X('is_residential:N',
            title='Property Type',
            axis=alt.Axis(labelExpr="datum.value ? 'Residential' : 'Non-Residential'")),
    y=alt.Y('land_value_pct:Q',
            title='Land Value as % of Total',
            scale=alt.Scale(domain=[0, 100])),
    color=alt.Color('is_residential:N',
                    legend=alt.Legend(title='Type'),
                    scale=alt.Scale(domain=[True, False], 
                                  range=['steelblue', 'orange'])),
    tooltip=[
        alt.Tooltip('is_residential:N', title='Residential'),
        alt.Tooltip('land_value_pct:Q', aggregate='median', title='Median Land %', format='.1f'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=400,
    height=350,
    title='Land Value Percentage: Residential vs Non-Residential'
).interactive()

chart_land_by_residential

### 5.4 Faceted Small Multiples

Create small multiples to compare distributions across multiple categories simultaneously.

In [ ]:
# Small Multiples 1: Value Distribution by Year (2019-2023)
df_recent = df_sample[df_sample['year'].between(2019, 2023)].copy()
df_recent_capped = df_recent[df_recent['total_assessed_value'] <= value_99th]

chart_value_by_year = alt.Chart(df_recent_capped).mark_bar(
    opacity=0.7
).encode(
    x=alt.X('total_assessed_value:Q',
            bin=alt.Bin(maxbins=30),
            title='Total Assessed Value ($)',
            axis=alt.Axis(format='$,.0s', labelAngle=-45)),
    y=alt.Y('count()',
            title='Count'),
    color=alt.Color('year:N',
                    legend=None),
    tooltip=[
        alt.Tooltip('year:N', title='Year'),
        alt.Tooltip('total_assessed_value:Q', bin=True, title='Value Range', format='$,.0f'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=180,
    height=150
).facet(
    facet=alt.Facet('year:N', title='Year'),
    columns=3
).properties(
    title='Property Value Distribution by Year (2019-2023)'
).resolve_scale(
    y='independent'
)

chart_value_by_year

In [ ]:
# Small Multiples 2: Building Age by Top 6 Neighborhoods
top_6_neighborhoods = df_clean['neighborhood'].value_counts().head(6).index.tolist()
df_top_nbhd = df_sample[df_sample['neighborhood'].isin(top_6_neighborhoods)]

chart_age_by_neighborhood = alt.Chart(df_top_nbhd).mark_bar(
    opacity=0.7,
    color='purple'
).encode(
    x=alt.X('building_age:Q',
            bin=alt.Bin(maxbins=20),
            title='Building Age',
            scale=alt.Scale(domain=[0, 150])),
    y=alt.Y('count()',
            title='Count'),
    tooltip=[
        alt.Tooltip('neighborhood:N', title='Neighborhood'),
        alt.Tooltip('building_age:Q', bin=True, title='Age Range'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=180,
    height=150
).facet(
    facet=alt.Facet('neighborhood:N', title=None),
    columns=3
).properties(
    title='Building Age Distribution by Top 6 Neighborhoods'
).resolve_scale(
    y='independent'
)

chart_age_by_neighborhood

In [ ]:
# Small Multiples 3: Bedrooms Distribution by Value Tier
df_bedrooms = df_sample[df_sample['number_of_bedrooms'] <= 5].copy()

chart_bedrooms_by_tier = alt.Chart(df_bedrooms).mark_bar(
    opacity=0.7
).encode(
    x=alt.X('number_of_bedrooms:O',
            title='Number of Bedrooms'),
    y=alt.Y('count()',
            title='Count'),
    color=alt.Color('value_tier:N',
                    legend=None),
    tooltip=[
        alt.Tooltip('value_tier:N', title='Value Tier'),
        alt.Tooltip('number_of_bedrooms:O', title='Bedrooms'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=150,
    height=150
).facet(
    facet=alt.Facet('value_tier:N', 
                    title='Value Tier',
                    sort=['Budget', 'Mid-range', 'High-end', 'Luxury']),
    columns=4
).properties(
    title='Bedroom Distribution by Value Tier'
).resolve_scale(
    y='independent'
)

chart_bedrooms_by_tier

### 5.5 Interactive Selection and Filtering

Create linked visualizations with interactive selection.

In [ ]:
# Useful when you want to focus on a subset and see statistical summaries

# Create separate brush for filtered version
brush_filter = alt.selection_interval(encodings=['x'])

# Same histogram
hist_filter = alt.Chart(df_sample).mark_bar(
    opacity=0.7
).encode(
    x=alt.X('building_age:Q',
            bin=alt.Bin(maxbins=30),
            title='Building Age (years)'),
    y=alt.Y('count()',
            title='Count'),
    color=alt.condition(brush_filter, 
                       alt.value('steelblue'), 
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('building_age:Q', bin=True, title='Age Range'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=600,
    height=200,
    title='Select Age Range (brush over bars)'
).add_params(
    brush_filter
)

# Scatterplot with filtering + count
scatter_filtered = alt.Chart(df_sample).mark_circle(
    size=40,
    opacity=0.6
).encode(
    x=alt.X('property_area:Q',
            title='Property Area (sqft)',
            scale=alt.Scale(type='log')),
    y=alt.Y('total_assessed_value:Q',
            title='Total Assessed Value ($)',
            scale=alt.Scale(type='log'),
            axis=alt.Axis(format='$,.0s')),
    color=alt.Color('building_age:Q', 
                   scale=alt.Scale(scheme='viridis'),
                   legend=alt.Legend(title='Building Age')),
    tooltip=[
        alt.Tooltip('building_age:Q', title='Building Age'),
        alt.Tooltip('property_area:Q', title='Property Area (sqft)', format=',.0f'),
        alt.Tooltip('total_assessed_value:Q', title='Value', format='$,.0f'),
        alt.Tooltip('neighborhood:N', title='Neighborhood')
    ]
).properties(
    width=600,
    height=250
).transform_filter(
    brush_filter
).interactive()

# Add a text annotation showing count of filtered points
text_count = alt.Chart(df_sample).mark_text(
    align='left',
    baseline='top',
    fontSize=12,
    dx=5,
    dy=5
).encode(
    text=alt.Text('count():Q', format=',.0f')
).transform_filter(
    brush_filter
).properties(
    width=600,
    height=30,
    title='Selected Properties Count'
)

# Combine all three
chart_filtered = alt.vconcat(
    hist_filter,
    text_count,
    scatter_filtered
).properties(
    title='Interactive Linked Views: Brush Selection (Filter Mode)'
)

chart_filtered

In [ ]:
# Interactive Selection 2: Click selection on neighborhoods

# Top 10 neighborhoods for clarity
top_10_neighborhoods = df_clean['neighborhood'].value_counts().head(10).index.tolist()
df_top10 = df_sample[df_sample['neighborhood'].isin(top_10_neighborhoods)]

# Create click selection
click = alt.selection_point(fields=['neighborhood'], empty=False)

# Bar chart of neighborhoods with click selection
nbhd_bars = alt.Chart(df_top10).mark_bar().encode(
    y=alt.Y('neighborhood:N',
            title='Neighborhood',
            sort='-x'),
    x=alt.X('count()',
            title='Number of Properties (in sample)'),
    color=alt.condition(click,
                       alt.Color('neighborhood:N', legend=None),
                       alt.value('lightgray')),
    tooltip=[
        alt.Tooltip('neighborhood:N', title='Neighborhood'),
        alt.Tooltip('count()', title='Count in Sample')
    ]
).properties(
    width=300,
    height=300,
    title='Click to Select Neighborhood'
).add_params(
    click
)

# Histogram filtered by neighborhood selection
value_hist_filtered = alt.Chart(df_top10).mark_bar(
    opacity=0.7
).encode(
    x=alt.X('total_assessed_value:Q',
            bin=alt.Bin(maxbins=30),
            title='Total Assessed Value ($)',
            axis=alt.Axis(format='$,.0s')),
    y=alt.Y('count()',
            title='Count'),
    color=alt.Color('neighborhood:N', legend=None),
    tooltip=[
        alt.Tooltip('neighborhood:N', title='Neighborhood'),
        alt.Tooltip('total_assessed_value:Q', bin=True, title='Value Range', format='$,.0f'),
        alt.Tooltip('count()', title='Count')
    ]
).properties(
    width=400,
    height=300,
    title='Value Distribution for Selected Neighborhood'
).transform_filter(
    click
).interactive()

# Combine horizontally
chart_click_selection = alt.hconcat(nbhd_bars, value_hist_filtered).properties(
    title='Interactive Click Selection: Neighborhood Filter'
)

chart_click_selection

### 5.6 Vega-Lite Summary

Summary of interactive visualizations created with Vega-Lite.

In [ ]:

viz_summary = {
    'Univariate Histograms': 3,
    'Boxplots': 3,
    'Faceted Small Multiples': 3,
    'Interactive Selections': 2
}

print("\nVisualizations Created:")
total = 0
for viz_type, count in viz_summary.items():
    print(f"  • {viz_type}: {count}")
    total += count


## 6. Final Dataset Summary

In [ ]:
# Display final dataset summary
print("FINAL DATASET SUMMARY")
print("="*80)

print(f"\nFinal dataset shape: {df_clean.shape}")
print(f"Memory usage: {df_clean.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Date range: {df_clean['year'].min()} - {df_clean['year'].max()}")
print(f"Number of neighborhoods: {df_clean['neighborhood'].nunique()}")

print("\nColumn List:")
for i, col in enumerate(df_clean.columns, 1):
    dtype = df_clean[col].dtype
    print(f"  {i:2d}. {col:40s} ({dtype})")

print("\nKey Statistics:")
print(f"  Total properties: {len(df_clean):,}")
print(f"  Median value: ${df_clean['total_assessed_value'].median():,.0f}")
print(f"  Residential properties: {df_clean['is_residential'].sum():,} ({df_clean['is_residential'].sum()/len(df_clean)*100:.1f}%)")

# Display sample of final dataset
print("\nSample of cleaned and transformed dataset:")
display(df_clean.head(10))

## 6. Save Processed Dataset

We'll export the cleaned dataset in multiple formats:
- **CSV**: Universal compatibility, human-readable
- **Parquet**: Efficient columnar storage, faster I/O
- **GeoJSON**: Geographic data for mapping applications
- **Pickle**: Python-specific, fastest loading

### 6.1 Verify Geographic Data

In [ ]:
# Verify we have geographic coordinates (already extracted from GeoJSON)
print("Verifying geographic coordinates...")

if 'latitude' in df_clean.columns and 'longitude' in df_clean.columns:
    valid_coords = df_clean[['latitude', 'longitude']].notna().all(axis=1).sum()
    print(f"✓ Coordinates available for {valid_coords:,} properties ({valid_coords/len(df_clean)*100:.1f}%)")
    
    # Show coordinate ranges (SF boundaries)
    if valid_coords > 0:
        lat_valid = df_clean['latitude'].dropna()
        lon_valid = df_clean['longitude'].dropna()
        print(f"\n  Latitude range: {lat_valid.min():.4f} to {lat_valid.max():.4f}")
        print(f"  Longitude range: {lon_valid.min():.4f} to {lon_valid.max():.4f}")
        print(f"  Expected SF bounds: Lat 37.7-37.8, Lon -122.5 to -122.4")
else:
    print("Warning: No coordinate columns found.")
    print("Creating placeholder coordinates for export...")
    df_clean['latitude'] = None
    df_clean['longitude'] = None

### 6.2 Export to CSV

In [ ]:
import os

# Save as CSV
csv_file = 'sf_property_data_clean.csv'
print("Saving to CSV format...")
df_clean.to_csv(csv_file, index=False)

if os.path.exists(csv_file):
    file_size = os.path.getsize(csv_file) / 1024**2
    print(f"✓ CSV saved: {csv_file}")
    print(f"  File size: {file_size:.1f} MB")
    print(f"  Rows: {len(df_clean):,}")
    print(f"  Columns: {len(df_clean.columns)}")

### 6.3 Export to Parquet

In [ ]:
# Save as Parquet (efficient columnar format)
parquet_file = 'sf_property_data_clean.parquet'
print("\nSaving to Parquet format...")

# Convert category types back to string for Parquet compatibility
df_parquet = df_clean.copy()
for col in df_parquet.select_dtypes(include=['category']).columns:
    df_parquet[col] = df_parquet[col].astype(str)

df_parquet.to_parquet(parquet_file, engine='pyarrow', compression='snappy', index=False)

if os.path.exists(parquet_file):
    file_size = os.path.getsize(parquet_file) / 1024**2
    print(f"✓ Parquet saved: {parquet_file}")
    print(f"  File size: {file_size:.1f} MB")
    print(f"  Compression: snappy")
    print(f"  Size reduction vs CSV: {(1 - file_size/(os.path.getsize(csv_file)/1024**2))*100:.1f}%")

### 6.4 Export to GeoJSON

We'll export the geographic data back to GeoJSON format for mapping applications.

In [ ]:
# Create GeoJSON for properties with valid coordinates
geojson_file = 'sf_property_data_geo.geojson'
print("Saving to GeoJSON format...")

# Filter to properties with valid coordinates
df_geo = df_clean[df_clean[['latitude', 'longitude']].notna().all(axis=1)].copy()

if len(df_geo) > 0:
    print(f"  Creating GeoDataFrame from {len(df_geo):,} properties with coordinates...")
    
    # Limit to reasonable sample size for GeoJSON (to keep file manageable)
    sample_size = min(50000, len(df_geo))
    df_geo_sample = df_geo.sample(n=sample_size, random_state=42) if len(df_geo) > sample_size else df_geo
    
    # Create Point geometries from coordinates
    from shapely.geometry import Point
    
    geometry = [Point(xy) for xy in zip(df_geo_sample['longitude'], df_geo_sample['latitude'])]
    
    # Create GeoDataFrame
    gdf_export = gpd.GeoDataFrame(df_geo_sample, geometry=geometry, crs="EPSG:4326")
    
    # Select key columns for export (avoid overly large file)
    export_columns = [
        'year', 'neighborhood', 'parcel_number',
        'total_assessed_value', 'assessed_land_value', 'assessed_improvement_value',
        'property_class_code_definition', 'number_of_bedrooms', 'number_of_bathrooms',
        'property_area', 'building_age', 'land_value_pct',
        'year_property_built', 'value_tier', 'is_residential',
        'geometry'
    ]
    
    # Filter to available columns
    available_export_cols = [col for col in export_columns if col in gdf_export.columns or col == 'geometry']
    gdf_export_subset = gdf_export[available_export_cols]
    gdf_export_subset = gdf_export_subset.astype({
        col: "float64" for col in gdf_export_subset.select_dtypes(include=["float16", "float32"]).columns
    })
    # Export to GeoJSON
    gdf_export_subset.to_file(geojson_file, driver='GeoJSON')
    
    if os.path.exists(geojson_file):
        file_size = os.path.getsize(geojson_file) / 1024**2
        print(f"✓ GeoJSON saved: {geojson_file}")
        print(f"  File size: {file_size:.1f} MB")
        print(f"  Features: {len(gdf_export_subset):,}")
        print(f"  Properties per feature: {len(available_export_cols) - 1}")  # -1 for geometry
        print(f"  CRS: EPSG:4326 (WGS84)")
        if len(df_geo) > sample_size:
            print(f"  Note: Sampled {sample_size:,} of {len(df_geo):,} properties with coordinates")
            print(f"        For full export, adjust sample_size in the code")
else:
    print("  Warning: No properties with valid coordinates found. GeoJSON not created.")
    print("  This may occur if the data source didn't include geographic coordinates.")

### 6.5 Save Pickle for Fast Loading

In [ ]:
# Save as pickle for fastest loading in Python
pickle_file = 'sf_property_data_clean.pkl'
print("\nSaving to Pickle format...")
df_clean.to_pickle(pickle_file)

if os.path.exists(pickle_file):
    file_size = os.path.getsize(pickle_file) / 1024**2
    print(f"✓ Pickle saved: {pickle_file}")
    print(f"  File size: {file_size:.1f} MB")

### 6.6 Export Summary

In [ ]:
# Summary of all exports
print("\n" + "="*80)
print("EXPORT SUMMARY")
print("="*80)

export_files = [
    ('CSV', csv_file, 'Universal compatibility, human-readable'),
    ('Parquet', parquet_file, 'Efficient columnar storage, ~10x faster than CSV'),
    ('GeoJSON', geojson_file, 'Geographic data for mapping (QGIS, Leaflet, etc.)'),
    ('Pickle', pickle_file, 'Python-specific, fastest loading')
]

for format_name, filename, description in export_files:
    if os.path.exists(filename):
        size_mb = os.path.getsize(filename) / 1024**2
        print(f"\n{format_name.upper()}:")
        print(f"  File: {filename}")
        print(f"  Size: {size_mb:.1f} MB")
        print(f"  Use: {description}")

---
